In [35]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

horizon = 24 

jobs = [
    {"duration": 1},                
    {"fixed_start": 8 , "duration": 1}, 
    {"fixed_start": 6 , "duration": 1},
    {"preferred_start": 6 , "duration": 1}, 
    {"preferred_start": 7 , "duration": 1} 
]
job_variables = {}

print(list(enumerate(jobs)))

for i, job in enumerate(jobs):
  if 'fixed_start' in job:
    start_time = model.NewIntVar(job['fixed_start'],job['fixed_start'],f"job{i}_start")
    print(job['fixed_start'])
  else:
    start_time = model.NewIntVar(0,horizon - job['duration'], f"job{i}_start")
  job_variables[i] = {
    "start_time": start_time,
    "duration" : job['duration']
    }
  
# job_end = [model.new_int_var(job_preference[i], job_preference[i]+duration, f"JobEnd{i}") for i in range (number_of_jobs)]
# model.add_all_different(job_end)
# print(job_end)

print(job_variables)

intervals = []
for i, job in enumerate(jobs):
    start_time = job_variables[i]['start_time']
    duration = job_variables[i]['duration']
    end_time = start_time + duration
    intervals.append(model.NewIntervalVar(start_time, duration, end_time, f"Job{i}_interval"))

model.AddNoOverlap(intervals)

makespan = model.NewIntVar(0 , horizon , 'makespan')

job_end_times = [job_variables[i]['start_time'] + job_variables[i]['duration'] for i in range(len(jobs))]

model.AddMaxEquality(makespan, job_end_times)

model.Minimize(makespan)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"schedule found {solver.ObjectiveValue()} hours")
    for i, job in enumerate(jobs):
        print(f"Job {i + 1} starts at {solver.Value(job_variables[i]['start_time'])} ")
else:
    print("No solution found.")





[(0, {'duration': 1}), (1, {'fixed_start': 8, 'duration': 1}), (2, {'fixed_start': 6, 'duration': 1}), (3, {'preferred_start': 6, 'duration': 1}), (4, {'preferred_start': 7, 'duration': 1})]
8
6
{0: {'start_time': job0_start(0..23), 'duration': 1}, 1: {'start_time': job1_start(8), 'duration': 1}, 2: {'start_time': job2_start(6), 'duration': 1}, 3: {'start_time': job3_start(0..23), 'duration': 1}, 4: {'start_time': job4_start(0..23), 'duration': 1}}
schedule found 9.0 hours
Job 1 starts at 0 
Job 2 starts at 8 
Job 3 starts at 6 
Job 4 starts at 1 
Job 5 starts at 2 
